In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import csv
from datetime import datetime
import re

chrome_options = Options()
chrome_options.add_argument('--start-maximized')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument('--disable-popup-blocking')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

driver.get("https://torecacamp-pokemon.com/")
print("Website loaded.")

# handle pop ups
time.sleep(4)
try:
    # close pop up if it exists
    close_button = driver.find_element(By.CSS_SELECTOR, ".popup-close-button")
    close_button.click()
    print("Popup closed.")
except:
    driver.refresh()
    time.sleep(3)
    print("Page refreshed.")

# csv files to incrementally save data
current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_filename = f"torecacamp_{current_datetime}.csv"

csv_file = open(csv_filename, 'w', newline='', encoding='utf-8-sig')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Category', 'Subcategory', 'Card Name', 'Price', 'URL'])

all_data = []

try:
    # find all main nav items through xpath
    nav_items = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "/html/body/div[3]/section/nav/div/div/ul/li"))
    )
    
    # only get items 3 to 8 (Scarlet and violet to Legend)
    for i in range(2, 8):
        if i >= len(nav_items):
            print(f"Nav item {i+1} doesn't exist. Skipping.")
            continue
            
        print(f"\nProcessing main nav item {i+1}")
        
        main_nav = nav_items[i]
        ActionChains(driver).move_to_element(main_nav).perform()
        time.sleep(1)
        
        # find all dropdown items for this nav through xpath
        try:
            dropdown_items = WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.XPATH, f"/html/body/div[3]/section/nav/div/div/ul/li[{i+1}]/ul/li"))
            )
            
            # only second dropdown item (expansion packs)
            if len(dropdown_items) >= 2:
                second_dropdown = dropdown_items[1]
                
                ActionChains(driver).move_to_element(second_dropdown).perform()
                time.sleep(1)
                
                # get all sub-dropdown items
                sub_dropdown_items = WebDriverWait(driver, 5).until(
                    EC.presence_of_all_elements_located((By.XPATH, f"/html/body/div[3]/section/nav/div/div/ul/li[{i+1}]/ul/li[2]/ul/li"))
                )
                
                main_category = main_nav.text.strip()
                subcategory = second_dropdown.text.strip()
                
                # process each sub-dropdown item
                for j, sub_item in enumerate(sub_dropdown_items):
                    try:
                        link = sub_item.find_element(By.TAG_NAME, "a")
                        link_url = link.get_attribute("href")
                        link_text = link.text.strip()
                        
                        print(f"  Clicking on: {link_text}")
                        
                        driver.execute_script("window.open(arguments[0]);", link_url)
                        
                        driver.switch_to.window(driver.window_handles[-1])
                        time.sleep(3)
                        
                        # scrape all pages of cards
                        page_num = 1
                        while True:
                            print(f"    Scraping page {page_num}")
                            
                            WebDriverWait(driver, 10).until(
                                EC.presence_of_element_located((By.CSS_SELECTOR, ".product-item__title"))
                            )
                            
                            cards = driver.find_elements(By.CSS_SELECTOR, ".product-item__title")
                            prices = driver.find_elements(By.CSS_SELECTOR, ".price-list span:not(.visually-hidden)")
                            
                            # process each card
                            for k in range(len(cards)):
                                if k < len(prices):
                                    card_name = cards[k].text.strip()
                                    card_price = prices[k].text.strip()
                                    
                                    csv_writer.writerow([main_category, subcategory, card_name, card_price, driver.current_url])
                                    
                                    # also keep in memory
                                    all_data.append({
                                        "Category": main_category,
                                        "Subcategory": subcategory,
                                        "Card Name": card_name,
                                        "Price": card_price,
                                        "URL": driver.current_url
                                    })
                            
                            # check for next page
                            try:
                                next_button = WebDriverWait(driver, 5).until(
                                    EC.presence_of_element_located((By.XPATH, "//a[@rel='next']"))
                                )
                                next_button.click()
                                page_num += 1
                                time.sleep(3)
                            except:
                                print("    No more pages.")
                                break
                        
                        driver.close()
                        driver.switch_to.window(driver.window_handles[0])
                        time.sleep(1)
                        
                        ActionChains(driver).move_to_element(main_nav).perform()
                        time.sleep(1)
                        
                        dropdown_items = WebDriverWait(driver, 5).until(
                            EC.presence_of_all_elements_located((By.XPATH, f"/html/body/div[3]/section/nav/div/div/ul/li[{i+1}]/ul/li"))
                        )
                        second_dropdown = dropdown_items[1]
                        ActionChains(driver).move_to_element(second_dropdown).perform()
                        time.sleep(1)
                        
                    except Exception as e:
                        print(f"    Error processing sub-item {j+1}: {str(e)}")
                        if len(driver.window_handles) > 1:
                            driver.close()
                            driver.switch_to.window(driver.window_handles[0])
            else:
                print(f"  No second dropdown item found for nav item {i+1}")
                
        except Exception as e:
            print(f"  Error processing dropdown for nav item {i+1}: {str(e)}")

except Exception as e:
    print(f"Error in main navigation process: {str(e)}")

csv_file.close()

df = pd.DataFrame(all_data)

driver.quit()
print(f"Scraping finished! Raw data saved to {csv_filename}")

df['Price'] = df['Price'].apply(lambda x: re.sub(r'[^\d,¥]+', '', x))

df_clean = df[['Card Name', 'Price']]

def extract_card_details(card_name):
    # Pattern 1: Standard format with 【状態X】
    pattern1 = re.compile(r'^【状態(?P<condition>[A-C][+-]?)】(?P<card_name>.+?)\s+(?P<set>[A-Z0-9]+)\s+(?P<card_number>\d+/\d+)(?:\s+【(?P<letter>[^】]+)】)?(?:\s+(?P<extra>.+))?$')
    
    # Pattern 2: Standard format without condition
    pattern2 = re.compile(r'^(?P<card_name>.+?)\s+(?P<set>[A-Z0-9]+)\s+(?P<card_number>\d+/\d+)(?:\s+【(?P<letter>[^】]+)】)?(?:\s+(?P<extra>.+))?$')
    
    # Pattern 3: Format with "状態X)" instead of 【状態X】
    pattern3 = re.compile(r'^状態(?P<condition>[A-C][+-]?)\)(?P<card_name>.+?)\s+(?P<set>[A-Z0-9]+)\s+(?P<card_number>\d+/\d+)(?:\s+(?P<extra>.+))?$')
    
    # Pattern 4: Format with 1ED at the end
    pattern4 = re.compile(r'^【状態(?P<condition>[A-C][+-]?)】(?P<card_name>.+?)\s+(?P<set>[A-Z0-9]+)\s+(?P<card_number>\d+/\d+)(?:\s+1ED)(?:\s+(?P<extra>.+))?$')
    
    # Pattern 5: Format without condition but with 1ED
    pattern5 = re.compile(r'^(?P<card_name>.+?)\s+(?P<set>[A-Z0-9]+)\s+(?P<card_number>\d+/\d+)(?:\s+1ED)(?:\s+(?P<extra>.+))?$')
    
    for pattern in [pattern1, pattern3, pattern4, pattern2, pattern5]:
        match = pattern.match(card_name)
        if match:
            d = match.groupdict()
            condition = d.get('condition', 'A')  # Default to 'A' if not specified
            letter = d.get('letter', None)
            extra = d.get('extra', None)
            
            return pd.Series([
                condition, 
                d['card_name'].strip(), 
                d['set'], 
                d['card_number'],
                letter,
                extra
            ])

    # fallback for unusual formats
    parts = card_name.split()
    if len(parts) >= 3:
        # Try to find the card number pattern (typically XXX/XXX)
        for i, part in enumerate(parts):
            if re.match(r'\d+/\d+', part):
                # Found a card number
                set_code = parts[i-1] if i > 0 else None
                
                # Check if there's a condition marker
                condition = None
                if card_name.startswith('【状態'):
                    condition_match = re.match(r'【状態([A-C][+-]?)】', card_name)
                    condition = condition_match.group(1) if condition_match else None
                
                # Everything before the set code (minus condition) is the card name
                name_parts = parts[:i-1]
                if condition and card_name.startswith('【状態'):
                    # Remove the condition part from name
                    full_name = ' '.join(name_parts)
                    card_name_clean = re.sub(r'^【状態[A-C][+-]?】\s*', '', full_name)
                else:
                    card_name_clean = ' '.join(name_parts)
                
                return pd.Series([
                    condition if condition else 'A',
                    card_name_clean.strip(),
                    set_code,
                    part,
                    None,
                    ' '.join(parts[i+1:]) if i+1 < len(parts) else None
                ])
    
    # If all else fails, return None for all fields
    return pd.Series([None, None, None, None, None, None])

new_cols = df_clean['Card Name'].apply(extract_card_details)
new_cols.columns = ['condition', 'card_name', 'set', 'card_number', 'letter', 'extra']

df_expanded = pd.concat([df_clean, new_cols], axis=1)

# Convert None to 'A' for condition where appropriate
df_expanded['condition'] = df_expanded['condition'].fillna('A')

df_expanded.to_csv(f"clean_torecacamp_{current_datetime}.csv", index=False)

Website loaded.
Page refreshed.

Processing main nav item 3
  Clicking on: SV10 / ロケット団の栄光
    Scraping page 1
    Scraping page 2
    Scraping page 3
    Scraping page 4
    Scraping page 5
    Scraping page 6
    Scraping page 7
    Scraping page 8
    Scraping page 9
    Scraping page 10
    No more pages.
  Clicking on: SV9a / 熱風のアリーナ
    Scraping page 1
    Scraping page 2
    Scraping page 3
    Scraping page 4
    Scraping page 5
    Scraping page 6
    Scraping page 7
    No more pages.
  Clicking on: SV9 / バトルパートナーズ
    Scraping page 1
    Scraping page 2
    Scraping page 3
    Scraping page 4
    Scraping page 5
    Scraping page 6
    Scraping page 7
    Scraping page 8
    Scraping page 9
    Scraping page 10
    No more pages.
  Clicking on: SV8a / テラスタルフェスex
    Scraping page 1
    Scraping page 2
    Scraping page 3
    Scraping page 4
    Scraping page 5
    Scraping page 6
    Scraping page 7
    Scraping page 8
    Scraping page 9
    Scraping page 10
    Scraping pa

ModuleNotFoundError: No module named 'openpyxl'